In [1]:
library(tidyverse)
library(lubridate)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


(1) Data Description

The data for this project comes from a research Minecraft server that tracked player profiles and gameplay activity. There are two datasets: players.csv and sessions.csv.

The players dataset contains 197 players and seven variables: experience (skill level), subscribe (newsletter sign-up), hashedEmail (anonymous identifier), played_hours (total playtime), and name, gender, and age. On average, players have spent about 17.26 hours in the game, though some have played much more or very little.

The sessions dataset includes 1,536 gameplay sessions with start and end times. From these, session duration in minutes can be calculated. Most sessions last around 30 minutes, but some have negative or extremely long durations, suggesting recording errors. Some sessions also have missing end times, possibly due to disconnections or server issues.

Both datasets are linked by hashedEmail, allowing player-level information to connect with gameplay behaviour. The data is valuable for studying engagement but requires cleaning before modeling.